In [1]:
import pyspiel
from open_spiel.python.observation import make_observation
from open_spiel.python.algorithms import get_all_states
from open_spiel.python.algorithms.alpha_zero import model as model_lib

In [2]:
game = pyspiel.load_game("leduc_poker")
states = get_all_states.get_all_states(game, to_string=str)
print(len(states))

9300


In [3]:
state = game.new_initial_state()
print(state.legal_actions())

[0, 1, 2, 3, 4, 5]


In [ ]:
state.apply_action(0)
state.apply_action(2)

In [ ]:
class player_observation_tree:
    
    def __init__(self, game, player_id):
        self.player = player_id
        
        # perfect_recall: if true, each observation must allow the observing player to
        #                 reconstruct their history of actions and observations.
        # public_info:    if true, the observation should include public information
        # private_info:   specifies for which players private information should be
        #                 included - all players, the observing player, or no players
        # we can set public_info to True while feeding into the NN to get predicted true state 
        self.observation = make_observation(
        game,
        pyspiel.IIGObservationType(
                                perfect_recall=False,
                                public_info=False,
                                private_info=pyspiel.PrivateInfoType.SINGLE_PLAYER))
        
        
        
    def get_private_observations(self, state):
        self.observation.set_from(state, player=self.player)
        return self.observation.tensor
    
    
class game_tree:
    
    def __init__(self, game):
        self.observation = make_observation(
        game,
        pyspiel.IIGObservationType(
                                perfect_recall=False,
                                public_info=True,
                                private_info=pyspiel.PrivateInfoType.ALL_PLAYERS))
        
        
    def get_all_childs(self, state):
        
        if (state.current_player() == 0):
            player = 0
        else:
            player = 1
        
        actions_child_states = {}
        for action in state.legal_actions():
            child_state = state.child(action)
            self.observation.set_from(child_state, player=player)
            actions_child_states[action] = self.observation.tensor
            
        return actions_child_states
    
    
class public_observation_tree:
    
    def __init__(self, game):
        
        self.observation = make_observation(
        game,
        pyspiel.IIGObservationType(
                                perfect_recall=False,
                                public_info=True,
                                private_info=pyspiel.PrivateInfoType.NONE))
        
        
        
    def get_public_observations(self,state):
        
        if (state.current_player() == 0):
            self.observation.set_from(state, player=0)
            return self.observation.tensor
        
        self.observation.set_from(state, player=1)
        return self.observation.tensor
    

In [ ]:
first_player_tree = player_observation_tree(game, 0)
print(first_player_tree.get_private_observations(state))

In [ ]:
second_player_tree = player_observation_tree(game, 1)
print(first_player_tree.get_private_observations(state))

In [ ]:
public_observation_tree = public_observation_tree(game)
print(public_observation_tree.get_public_observations(state))

In [ ]:
game_tree = game_tree(game)
print(game_tree.get_all_childs(state))


In [ ]:
first_player_tree = player_observation_tree(game, 0)
a = first_player_tree.get_private_observations(state)
print(a.shape[0])
print(game.observation_tensor_shape())

In [ ]:
model_saves_path_1 = '../model_saves/poker/state_model'
model_saves_path_2 = '../model_saves/poker/observation_model'

state_model = model_lib.Model.build_model(
      "mlp", game.observation_tensor_shape(), game.num_distinct_actions(),
      nn_width=10, nn_depth=5, weight_decay=0.0001, learning_rate=0.001, path=model_saves_path_1)


observation_model = model_lib.Model.build_model(
      "mlp", [a.shape[0]], game.num_distinct_actions(),
      nn_width=10, nn_depth=5, weight_decay=0.0001, learning_rate=0.001, path=model_saves_path_2)

In [ ]:
print(state.current_player())

# here we clone the state to guess the game state for other player
state_clone = state.clone()

# player 0 plays the action in the start
value, policy = state_model.inference([state.observation_tensor()], [state.legal_actions_mask()])
print(value)
print(policy)
state.apply_action(1)
print(state.observation_tensor())

In [ ]:
print(state.current_player())
print(state_clone.legal_actions())

# we get the private observation for the player 1 using true state
b = second_player_tree.get_private_observations(state)

# we pass the private observation and state_clone legal masks
value, policy = observation_model.inference([b], [state_clone.legal_actions_mask()])
print(value)
print(policy)

In [ ]:
# we guess the state by applying the action
guess_state = state_clone.child(1)

# just checking whether guess state observations and true state observations are same or not
print(guess_state.observation_tensor())